### Scenario analysis result evaluation based on different criteria

#### Import libraries

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import os
import pandas as pd
import seaborn as sns

In [ ]:
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

#### Load the scenario analysis data

In [ ]:
file_path = "../reports/"
file_name = "Scenario_Analysis_All_Locations_All_Criteria.csv"

scenario_analysis_data = pd.read_csv(os.path.join(file_path, file_name))

#### Define the criteria selection

In [ ]:
# criteria_selection = ["fish_stock_health", "potential_habitat_restoration"]
# types = [1, 1]
# criteria_selection = ["average_income", "unemployment_rate"]
# types = [1, -1]
criteria_selection = ["marine_biodiversity", "carbon_sequestration_potential"]
types = [1, 1]

filter_columns = criteria_selection.copy()
filter_columns.append("distance_from_offshore_wind_farm")

#### Visualize the results of scenario analysis

In [ ]:
FONT_SIZE = 20

In [ ]:
scenario_analysis_results = {}
for community in list(scenario_analysis_data["community_name"].unique()):
    community_data = scenario_analysis_data[scenario_analysis_data["community_name"] == community].sort_values("distance_from_offshore_wind_farm", ascending=False)
    community_data = community_data[filter_columns]
    
    locations = list(community_data["distance_from_offshore_wind_farm"])
    drop_idxs = []
    for idx in range(len(locations) - 1):
        if locations[idx] == locations[idx+1]:
            if np.sum(list(community_data.iloc[idx][criteria_selection])) < np.sum(list(community_data.iloc[idx+1][criteria_selection])):
                drop_idxs.append(idx)
            else:
                drop_idxs.append(idx+1)
    drop_indexes = community_data.index[drop_idxs]
    community_data.drop(drop_indexes, inplace=True)
    scenario_analysis_results[community] = community_data.copy()
    
    # Plot impact ranges for selected criteria
    sorted_locations = list(community_data["distance_from_offshore_wind_farm"])
    sorted_locations = [str(x) for x in sorted_locations]
    
    palette = sns.color_palette("tab20c", len(criteria_selection))
    height = 0.35
    alpha = 1.0
    
    plt.figure(figsize=(20, 15))
    for criterion_idx, criterion in enumerate(criteria_selection):
        sorted_criterion_data = list(community_data[criterion])
        for i in range(len(community_data)):
            y = i - height * criterion_idx
            if sorted_criterion_data[i] < 0:
                plt.barh(y, sorted_criterion_data[i], height, left=0, color=palette[criterion_idx], alpha=alpha, label=criterion)
            else:
                plt.barh(y, sorted_criterion_data[i], height, left=0, color=palette[criterion_idx], alpha=alpha, label=criterion)
    plt.axvline(0, color="gray", linestyle='--', label="dataset average")
    plt.xlabel("Deviation from average [%]")
    plt.ylabel("Distance from shore [km]")
    plt.title(f"Scenario Analysis for Location {community}")
    plt.yticks(np.arange(len(sorted_locations))-height, sorted_locations)
    handles, labels = plt.gca().get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    plt.legend(by_label.values(), by_label.keys())
    plt.grid(axis='x', linestyle='--', alpha=0.5)

    plt.rc('font', size=FONT_SIZE)          # controls default text sizes
    plt.rc('axes', titlesize=FONT_SIZE)     # fontsize of the axes title
    plt.rc('axes', labelsize=FONT_SIZE)    # fontsize of the x and y labels
    plt.rc('xtick', labelsize=FONT_SIZE)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=FONT_SIZE)    # fontsize of the tick labels
    plt.rc('legend', fontsize=FONT_SIZE)    # legend fontsize
    plt.rc('figure', titlesize=FONT_SIZE)  # fontsize of the figure title
    
    plt.savefig(f"Scenario Analysis for Location {community} and Criteria {criteria_selection}.png")
    plt.close()

#### Find optimal location for each community based on selected criteria

In [ ]:
optimal_locations = {}
for community in scenario_analysis_results.keys():
    criteria_data = scenario_analysis_results[community][criteria_selection].to_numpy().transpose()
    types = np.array(types)
    criteria_data = criteria_data * types[:, np.newaxis]
    optimal_location_idx = np.argmax(criteria_data.sum(axis=0))
    optimal_locations[community] = scenario_analysis_results[community]["distance_from_offshore_wind_farm"].iloc[optimal_location_idx]

for community, location in optimal_locations.items():
    print(f"Optimal location for {community} is {location} km from shore.")

#### Save results to a file

In [ ]:
result = pd.concat([df.assign(Key=key) for key, df in scenario_analysis_results.items()])
result.to_csv(f"Scenario_Analysis_for_Criteria {criteria_selection}.csv")